In [ ]:
import six
import sys
if sys.version_info >= (3, 12, 0):
    sys.modules['kafka.vendor.six.moves'] = six.moves

In [ ]:
from kafka import KafkaProducer
import pandas as pd
import time
import random
import json

In [ ]:
df = pd.read_csv('../data/User0_credit_card_transactions.csv')
df.head()

In [ ]:
# Khởi tạo Kafka producer
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',  # Địa chỉ Kafka broker
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)


In [ ]:
# Topic Kafka
topic = 'Project'

In [ ]:
def clean_amount(amount):
    if '(' in amount:
        amount = amount.replace('(','')
        amount = amount.replace(')','')
        amount = amount.replace(',','')
        return -float(amount.replace('$',''))
    else:
        amount = amount.replace(',','') 
        return float(amount.replace('$',''))

In [ ]:
for index, row in df.iterrows():
    transaction = {
        "User": row['User'],
        "Card": row['Card'],
        "Year": row['Year'],
        "Month": row['Month'],
        "Day": row['Day'],
        "Time": row['Time'],
        "Amount": clean_amount(row['Amount']),
        "Use Chip": row['Use Chip'],
        "Merchant Name": row['Merchant Name'],
        "Merchant City": row['Merchant City'],
        "Merchant State": row['Merchant State'],
        "Zip": row['Zip'],
        "MCC": row['MCC'],
        "Errors?": row['Errors?'],
        "Is Fraud?": row['Is Fraud?']
    }
    
    # Gửi dữ liệu vào Kafka
    print(transaction)
    producer.send(topic, transaction)
    
    # Chờ thời gian ngẫu nhiên giữa 1s và 3s
    time.sleep(random.uniform(1, 3))

In [ ]:
producer.flush()
producer.close()